In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.model_selection import train_test_split

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariarousset/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dariarousset/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dariarousset/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
from nltk.corpus import stopwords
stop = set(stopwords.words('french'))

In [3]:
os.listdir('../raw_data/')

['daria_logreg_frenchtweets.ipynb',
 'ml_model_nb_allo.pkl',
 'xav_Deep_Learning_CNN.ipynb',
 'xav_Machine_Learning.ipynb',
 'Deep_Learning_CNN.ipynb',
 'twitter_api.ipynb',
 'processing(fr).ipynb',
 'xav_Deep_Learning.ipynb',
 'data_preprocessing-2.ipynb',
 '.keep',
 'daria_nb_frenchtweets.ipynb',
 'RNN-2.ipynb',
 'daria_bert.ipynb',
 'daria_logreg_frenchtweets_copy.ipynb',
 '.ipynb_checkpoints',
 'daria_nb_allocine.ipynb',
 'ml_model_nb.pkl',
 'daria_RNN.ipynb',
 'xav_Preprocessing twitter.ipynb']

In [13]:
french = pd.read_csv('../raw_data/allo_clean.csv').sample(10000).reset_index(drop=True)
french.polarity.value_counts()

0    5029
1    4971
Name: polarity, dtype: int64

In [14]:
X= french['clean_text']
y= french['polarity']

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

param_grid = {
     'tfidf__lowercase': (True, False),
     'tfidf__ngram_range': [(1, 1), (1, 2)],
     'tfidf__max_df': [0.60, 0.65, 0.70, 0.75, 0.85, 1],
     'clf__C': np.logspace(-4, 4, 10),
}

tfidf_clf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop)),
    ('clf', LogisticRegression(penalty='l2', n_jobs=-1, verbose=1)),
])

grid_search = GridSearchCV(
    tfidf_clf, param_grid, cv=10, 
    scoring='f1', return_train_score=True, 
    n_jobs=-1, verbose=1
)

grid_search.fit(X, y)

Fitting 10 folds for each of 240 candidates, totalling 2400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.0s finished


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words={'ai', 'aie',
                                                                    'aient',
                                                                    'aies',
                                                                    'ait', 'as',
                                                                    'au',
                                                                    'aura',
                                                                    'aurai',
                                                                    'auraient',
                                                                    'aurais',
                                                                    'aurait',
                                                                    'auras',
                                                                    'aurez',

In [16]:
grid_search.best_params_

{'clf__C': 10000.0,
 'tfidf__lowercase': True,
 'tfidf__max_df': 0.6,
 'tfidf__ngram_range': (1, 2)}

In [14]:
grid_search.best_score_

0.7654843272647883

In [15]:
predict = pd.read_csv("tweet_psg.csv")

In [17]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import TweetTokenizer
import spacy
stop = set(stopwords.words('french'))
nlp=spacy.load("fr_core_news_md")


def clean_text(text):
    #emojis=demoji.findall(text)
    #if emojis != {}:
        #for key,value in emojis.items(): 
            #if key in text:
                #try:
                    #translated_text=ts.translate_html(value, translator=ts.google, to_language='fr', n_jobs=-1)
                    #text=text.replace(key,translated_text)
                #except TypeError:
                    #pass
    # lower text
    text = text.lower()
    # remove puncutation
    for punctuation in string.punctuation.replace('#',''):
        text = text.replace(punctuation, ' ')
    # remove words that contain numbers
    text = ''.join(letter for letter in text if not letter.isdigit())
    #tokenization + remove stop words
    doc=nlp(text)
    lemmatized= [token.lemma_ for token in doc]
    # join all
    text = " ".join(lemmatized)
    return text

In [18]:
predict["clean_text"] = predict["tweet"].apply(clean_text)

In [19]:
X_test = predict['clean_text']

In [20]:
prediction = pd.DataFrame(grid_search.best_estimator_.predict(X_test))

In [21]:
real_twits_prediction = pd.concat([predict,prediction], axis=1)

In [22]:
real_twits_prediction.to_csv("prediction_reg.csv")